Entidades:
Libro: Tiene un título, un autor, un número de identificación único y un estado de disponibilidad (prestado o disponible).

Revista: Similar al libro, con un título, un número de identificación único y un estado de disponibilidad.

Miembro: Representa a los usuarios de la biblioteca. Tiene un nombre, un número de identificación único y puede tener un estado especial si es miembro del staff.

Préstamo: Registra la información de los préstamos, incluyendo el artículo prestado, el miembro que lo ha tomado prestado, la fecha de préstamo y la fecha de vencimiento.

Funcionalidades:
Prestar artículo: Verificar la disponibilidad del artículo y realizar el préstamo, actualizando su estado y registrando la información del préstamo.

Devolver artículo: Registrar el retorno del artículo y actualizar su estado de disponibilidad.

Gestionar reservas: Para los libros reservados, gestionar el préstamo a corto plazo y las restricciones de los miembros del staff.

Registrar préstamos y devoluciones: Mantener un registro de cuándo se toman prestados los artículos y cuándo se devuelven.

Consideraciones adicionales:
Se necesita un sistema de control de fechas para calcular la fecha de vencimiento de los préstamos.

Se debe implementar una restricción en el número de artículos que un miembro puede tener en préstamo simultáneamente, dependiendo de si es miembro del staff o no.

Los miembros del staff deben tener privilegios especiales para tomar prestadas las revistas.

In [ ]:
from datetime import datetime, timedelta # importa las clases datetime y timedelta del módulo datetime,fechas y tiempos en Python.

class Articulo:
    def __init__(self, id, titulo):#atributos como ID, título y disponibilidad.
        self.id = id
        self.titulo = titulo
        self.disponible = True

class Libro(Articulo):#no tienen ningún método adicional
    pass

class Revista(Articulo):#no tienen ningún método adicional
    pass

class Miembro:#miembro de la biblioteca, 
    def __init__(self, id, nombre, es_staff=False):
        self.id = id
        self.nombre = nombre
        self.es_staff = es_staff

    #el es_staff  es una forma de proporcionar un valor predeterminado para el atributo es_staff en caso de que no se especifique 
    # al crear una instancia de la clase Miembro. Esto significa que, si no se 
    # proporciona un valor para es_staff al crear un nuevo objeto Miembro, 
    # se asumirá automáticamente que el miembro no es del personal (es_staff=False).
    #si un miembro de la biblioteca es del personal administrativo o no.

class Prestamo:#un préstamo de un artículo a un miembro de la biblioteca. 
    def __init__(self, articulo, miembro):
        self.articulo = articulo
        self.miembro = miembro
        self.fecha_inicio = datetime.now()
        self.fecha_fin = self.fecha_inicio + timedelta(weeks=3)  # 3 semanas de préstamo

class ControlFechas:
    @staticmethod
    def verificar_vencimiento(prestamo):
        return prestamo.fecha_fin < datetime.now()#devuelve True si el préstamo ha vencido, y False de lo contrario.

class Biblioteca:# métodos para agregar artículos y miembros, prestar y devolver artículos, y verificar el vencimiento de los préstamos.
    def __init__(self):
        self.articulos = []
        self.miembros = []
        self.prestamos = []

    def agregar_articulo(self, articulo):
        self.articulos.append(articulo)

    def agregar_miembro(self, miembro):
        self.miembros.append(miembro)

    def prestar_articulo(self, articulo_id, miembro_id):
        articulo = next((a for a in self.articulos if a.id == articulo_id), None)
        miembro = next((m for m in self.miembros if m.id == miembro_id), None)
        if articulo and miembro:
            if isinstance(articulo, Revista):
                if miembro.es_staff:
                    prestamo = Prestamo(articulo, miembro)
                    self.prestamos.append(prestamo)
                    articulo.disponible = False
                    print(f"La revista {articulo.titulo} ha sido prestada a {miembro.nombre}.")
                else:
                    print("¡Solo el personal puede tomar prestadas revistas!")
            else:
                if len([p for p in self.prestamos if p.miembro == miembro]) < (12 if miembro.es_staff else 6):
                    prestamo = Prestamo(articulo, miembro)
                    self.prestamos.append(prestamo)
                    articulo.disponible = False
                    print(f"El artículo {articulo.titulo} ha sido prestado a {miembro.nombre}.")
                else:
                    print("El miembro ha alcanzado el límite de préstamos.")

    def devolver_articulo(self, articulo_id):
        prestamo = next((p for p in self.prestamos if p.articulo.id == articulo_id and p.articulo.disponible == False), None)
        if prestamo:
            prestamo.articulo.disponible = True
            print(f"El artículo {prestamo.articulo.titulo} ha sido devuelto.")
            self.prestamos.remove(prestamo)
        else:
            print("No se encontró un préstamo activo para el artículo especificado.")

# Ejemplo de uso:
biblioteca = Biblioteca()# instancia de la clase Biblioteca llamada biblioteca.

#crear objetos para ensayar
libro1 = Libro(1, "Harry Potter")
revista1 = Revista(2, "National Geographic")
miembro1 = Miembro(1, "Juan")
staff1 = Miembro(2, "María", es_staff=True)

biblioteca.agregar_articulo(libro1)
biblioteca.agregar_articulo(revista1)
biblioteca.agregar_miembro(miembro1)
biblioteca.agregar_miembro(staff1)

biblioteca.prestar_articulo(1, 1)  # Juan toma prestado un libro
biblioteca.prestar_articulo(2, 1)  # Juan intenta tomar prestada una revista
biblioteca.prestar_articulo(2, 2)  # María toma prestada una revista
biblioteca.prestar_articulo(1, 1)  # Juan intenta tomar prestado otro libro

biblioteca.devolver_articulo(1)  # Juan devuelve el libro

# Verificar vencimiento de préstamos
for prestamo in biblioteca.prestamos:
    if ControlFechas.verificar_vencimiento(prestamo):
        print(f"¡El préstamo del artículo {prestamo.articulo.titulo} ha vencido!")
